In [ ]:
#Basic Library 
import numpy as np 
import pandas as pd 
import math
import scipy 

#Visualization
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")
%matplotlib inline
import matplotlib.gridspec as gridspec
import cufflinks as cf
cf.go_offline()
from IPython.display import display
from PIL import Image

import warnings
warnings.filterwarnings("ignore")

In [ ]:
pip install feature-engine

# Greetting and Description
A Quick Summary .. this is the 3rd out of 6 begginer Notebooks
* I am unsing Machine Learning Algorithms such as ***Linear Regression, Decision Tree, Random Forest , Xgboost*** , I also  used Cross_Score to figure out the best algorithms , and finally tunning Parameter using ***GridSearchCv and  RandomizedSearchCV***
* Another Important Library is feature-engine Which ***named the columns*** After applying Onehotencoder
* Following Data Science Standad I am /using OSEMN Methodology to have a proper workflow
* Please Feel Free to leave Any Feedback

In [ ]:
path="../input/imagenes/osemn.jpeg"
display(Image.open(path))

# 1. Obtain Data

1.1 General View

In [ ]:
df = pd.read_csv('../input/big-mart-sales/train_v9rqX0R.csv')
print("---------Technocal Information-------------")
print("Data Set shape = {}".format(df.shape))
print('Data set Memory Usage = {:.2f} MB'.format(df.memory_usage().sum()/1024**2))
print("Data Colums type\n{}".format(df.dtypes.value_counts()))
df.describe()

# 2. Scrub Data

2.1 General View

In [ ]:
#General view
print('Target columns is = {}'.format(df.columns[-1]))
print("---------------")
print("Variable Target = 'Showing 10 only out of 1559' \n{}".format(df['Item_Identifier'].value_counts().index.to_list()[0:10]))
print('-------------')
print('Columns in df are ={}'.format(df.columns.to_list()))
print("\n----- Null Values ------\n")

# Missing Values
print(df.isnull().sum())

In [ ]:
print("Data Set")
df.head(5)

### **Notes:**

1. This is Regression challenge in which we have to predict the ***Total Sales***
2. Almost 4,000 ***Missing/null/nan*** values

#### ***Approach*** 
1. ***Identify any relation between Variables*** to fill those mising values .

In [ ]:
msno.matrix(df, sparkline=True, figsize=(10,5), sort='ascending', fontsize=12, labels=True, color=(0.25, 0.45, 0.6))
print("Based on this Visualization we can assume that NAN values are in the same row, now we will show those missing values using pandas and find any relation")

In [ ]:
 df[df.isna().any(axis=1)]

In [ ]:
def corr (df):
    
    correlation = df.corr()
    f,ax= plt.subplots(figsize = (15,10))
    mask = np.triu(correlation)
    sns.heatmap(correlation, annot=True, mask=mask,ax=ax,cmap='viridis')
    bottom,top = ax.get_ylim()
    ax.set_ylim(bottom+ 0.5, top - 0.5)
    
corr(df)

2.2 Missing Values

In [ ]:
for i in df.columns:
    value = df[i].isnull().sum()
    if value >0:
        print(i , "Missing Values",value)

2.3 Filling ***Weight***
* I logic is the Following group the set by Item which has unique,Size and bring the Mean weight then use this value to replace the Nan values 

In [ ]:
# Finding Item_Identifier, item Weight and Oulet Size (no null)

ndf = df[(df['Item_Weight'].notnull())&(df['Outlet_Size'].notnull())] #no null in the whole Data Set
dict_ndf = ndf.groupby(['Item_Identifier','Outlet_Size'])['Item_Weight'].mean().to_dict()


def finding_weights (col):
    model = col[0]
    measure = col[1]
    weight = col[2]
    
    for k, v in dict_ndf.items():
        if (k[0]==model) & (k[1]== measure):
            weight = v
        else :
            weight = weight 
            
    return weight

# Replacing Values 

df['Item_Weight']= df[['Item_Identifier','Outlet_Size','Item_Weight']].apply(finding_weights,axis = 1)
df.head(3)

2.4 Filling ***Outlet_size***

* In my understanding Each Item has unique Code, so I Create a list of unique Item-Size and the comparing them with the Item which has Nan values in Outlet-size

In [ ]:
#Finding Index and Replacing Grcoery 
grocery_index = df[(df['Outlet_Size'].isnull())&(df['Outlet_Type'] == 'Grocery Store')].index
df.loc[grocery_index,'Outlet_Size']="Small"

#Creating a List to Filter item 
Out_no_NAN = df.loc[(df['Outlet_Size'].notna())&(df['Outlet_Size'] =='Small')][['Item_Identifier','Outlet_Size']].value_counts().to_frame().reset_index()
values = list(zip(Out_no_NAN.Item_Identifier, Out_no_NAN.Outlet_Size))


def finding_size (col):
    item = col[0]
    outsize = col[1]
    
    if pd.isnull(outsize) :
        
        for i in values:
            if i[0] == item:
                outsize = i[1]
        return outsize 

    else:
        return outsize
    
# Replacing Values
df['Outlet_Size']= df[['Item_Identifier','Outlet_Size']].apply(finding_size,axis = 1)


print("Replacing Values Under 2 Condition- 1.Item name, 2. Outlet_size\nReplacing..............Done..!!!\n")
print("After Checking Out_size and Grouping by Outlet_Type I can noticed that All outlet_size in Grocery Store are small\nReplacing Nan for Small in outlet_size for Grocery Store......Done..!!")
df.head(5)

In [ ]:
for i in df.columns:
    value = df[i].isnull().sum()
    if value >0:
        print(i , "Missing Values",value)

In [ ]:
# Index
index_weight = df.loc[df['Item_Weight'].isna()].index
index_size = df.loc[df['Outlet_Size'].isna()].index

#Dropping 

df.drop(index_weight, inplace = True)
df.reset_index(drop=True)

df.drop(index_size, inplace = True)
df.reset_index(drop=True)

2.5 LF to Low Fat & reg to Regular

In [ ]:
def changing(x):
    
    if x == 'LF':
        x = 'Low Fat'
    elif x == 'low fat':
        x = 'Low Fat'
    elif x == 'reg':
        x = 'Regular'
    else:
        x=x
    return x

df['Item_Fat_Content'] = df['Item_Fat_Content'].apply(lambda x: changing(x))

df.head(3)

# 3. Explore Data

3.1 Exploration, Segmentation, Total Sales, Total Weight, Best Performance (Location) and more 

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(20,10))
grid = gridspec.GridSpec(ncols=6, nrows=2, figure=fig)



#bar plot Horizontal
ax1 = fig.add_subplot(grid[0, :2])
ax1.set_title('Type of Product per Store')
sns.countplot(y='Outlet_Type',hue = 'Item_Fat_Content',data = df, ax=ax1,) #Outlet Type

#bar plot Vertical
ax2 = fig.add_subplot(grid[1, :2])
ax2.set_title('Type of Products')
bar = sns.countplot(x='Item_Type', data = df, ax = ax2)
bar.set_xticklabels(bar.get_xticklabels(),  rotation=90, horizontalalignment='right') #Type of Products

#box plot Sales
ax3 = fig.add_subplot(grid[:, 2])
ax3.set_title('Total Sales')
sns.boxplot(df.loc[:,'Item_Outlet_Sales'], orient='v', ax = ax3) #Total Sales

#box plot Monthly payment
ax4 = fig.add_subplot(grid[:,3])
ax4.set_title("Weight")
sns.boxplot(df['Item_Weight'], orient='v' ,ax=ax4)


#-----------> Displot Distribution
ax5 = fig.add_subplot(grid[0, 4:6])
ax5.set_title("Sales by Store")
sns.barplot(x='Outlet_Type', y='Item_Outlet_Sales',data=df,ax=ax5) 


#-----------> Displot Distribution
ax6 = fig.add_subplot(grid[1, 4:6])
ax6.set_title("Sale Distribution, ST1='Orange',ST2 ='Green',ST3='Blue'")
SuperT1 = df[df['Outlet_Type']=='Supermarket Type1']
SuperT2 = df[df['Outlet_Type']=='Supermarket Type2']
SuperT3 = df[df['Outlet_Type']=='Supermarket Type3']
Grocery = df[df['Outlet_Type']=='Grocery Store']

sns.distplot(SuperT1['Item_Outlet_Sales'], color = 'orange', ax=ax6)
sns.distplot(SuperT2['Item_Outlet_Sales'], color = 'Green', ax=ax6)
sns.distplot(SuperT3['Item_Outlet_Sales'], color = 'Blue', ax=ax6)

plt.show()



In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(20,12))
grid = gridspec.GridSpec(ncols=2, nrows=2, figure=fig)

ax1 = fig.add_subplot(grid[0:, :1])
ax1.set_title('Product distribution per store')
distr=sns.swarmplot(x='Outlet_Type', y='Item_Outlet_Sales', hue='Item_Type' ,data=df,ax=ax1)
distr.set_xticklabels(distr.get_xticklabels(),  rotation=90, horizontalalignment='right')

ax2 = fig.add_subplot(grid[0, 1:])
ax2.set_title('Relation Sales~ Visibility per Product')
sns.scatterplot(y='Item_Outlet_Sales' ,x= 'Item_Visibility',  hue = 'Item_Type', data =df ,cmap='Spectral')

ax3 = fig.add_subplot(grid[1, 1:])
ax3.set_title('Relation Sales~ MRP per Product')
sns.scatterplot(y ='Item_Outlet_Sales', x = 'Item_MRP',data= df, hue = 'Outlet_Type', ax=ax3)
plt.show()


print("Low Fat is the category that performance better in sales , no matter the Store")
print("Top 5 items : Fruit and vegetables, Snack Foods, Baking food, Frozen foods , Dairy")
print("Average Sale in this data ser was almost 2000")
print("Average Weight 12.5")
print("Supermarket Type 3 perfomance better in sales ")
print("In ST1(Green) seems to sell many product which the price range is 0~2500\nIn ST3(Blue) seems to sell many product which the price range is 2500~7500")
print("The most expensive items are in ST1, however the one who has more products in ST1")
print("There is no relation between Sales and Visibility but there is between MRP and Sales")



2.2 Polloting After Normalization

In [ ]:
# Transforming 'Sale Price' using log1p
'''
df["Item_Outlet_Sales"] = np.log1p(df["Item_Outlet_Sales"])

fig = plt.figure(constrained_layout=True, figsize=(20,10))
grid = gridspec.GridSpec(ncols=6, nrows=2, figure=fig)



#bar plot Horizontal
ax1 = fig.add_subplot(grid[0, :2])
ax1.set_title('Type of Product per Store')
sns.countplot(y='Outlet_Type',hue = 'Item_Fat_Content',data = df, ax=ax1,) #Outlet Type

#bar plot Vertical
ax2 = fig.add_subplot(grid[1, :2])
ax2.set_title('Type of Products')
bar = sns.countplot(x='Item_Type', data = df, ax = ax2)
bar.set_xticklabels(bar.get_xticklabels(),  rotation=90, horizontalalignment='right') #Type of Products

#box plot Sales
ax3 = fig.add_subplot(grid[:, 2])
ax3.set_title('Total Sales')
sns.boxplot(df.loc[:,'Item_Outlet_Sales'], orient='v', ax = ax3) #Total Sales

#box plot Monthly payment
ax4 = fig.add_subplot(grid[:,3])
ax4.set_title("Weight")
sns.boxplot(df['Item_Weight'], orient='v' ,ax=ax4)


#-----------> Displot Distribution
ax5 = fig.add_subplot(grid[0, 4:6])
ax5.set_title("Sales by Store")
sns.barplot(x='Outlet_Type', y='Item_Outlet_Sales',data=df,ax=ax5) 


#-----------> Displot Distribution
ax6 = fig.add_subplot(grid[1, 4:6])
ax6.set_title("Sale Distribution, ST1='Orange',ST2 ='Green',ST3='Blue'")
SuperT1 = df[df['Outlet_Type']=='Supermarket Type1']
SuperT2 = df[df['Outlet_Type']=='Supermarket Type2']
SuperT3 = df[df['Outlet_Type']=='Supermarket Type3']
Grocery = df[df['Outlet_Type']=='Grocery Store']

sns.distplot(SuperT1['Item_Outlet_Sales'], color = 'orange', ax=ax6)
sns.distplot(SuperT2['Item_Outlet_Sales'], color = 'Green', ax=ax6)
sns.distplot(SuperT3['Item_Outlet_Sales'], color = 'Blue', ax=ax6)

plt.show()

print("After Standarization\n") '''




# 4. Model Data

In [ ]:
#Applying Machine Learning 
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV,cross_val_score
from sklearn.preprocessing import MinMaxScaler,LabelEncoder,OneHotEncoder
from feature_engine.categorical_encoders import OneHotCategoricalEncoder
from sklearn.metrics import mean_squared_error


4.1 Outliers


In [ ]:
#Fuction to find Outliers
def Outliers (data,column):
    mean_ = data[column].mean()
    Sdev_= data[column].std()
    Upper_limit= mean_+ (3*Sdev_)
    lower_limit= mean_- (3*Sdev_) #error 
    out= data[(data[column]>Upper_limit)|(data[column]<lower_limit)].index
    
       
    data.drop(out ,inplace=True)
    data.reset_index(drop= True)
    print(out)


#Dropping Colums 

df.drop(['Item_Identifier','Outlet_Identifier'], axis = 1 , inplace= True)
Outliers(df,"Item_Outlet_Sales")
df.head(5)

4.2. Label Encoding and OneHot Encoding(Using, feature-engine)

In [ ]:
#Label Encoding for Outlet_Size
LE = LabelEncoder()
df['Outlet_Size']=LE.fit_transform(df['Outlet_Size'])

#Encoding item_Fat Content
En = pd.get_dummies(df['Item_Fat_Content'], drop_first = True, prefix = 'LF')

#Encoding Other Variables 
OHC_ENG = OneHotCategoricalEncoder(top_categories = None,
                                  variables=['Item_Type','Outlet_Type','Outlet_Location_Type'],drop_last= False)
                                  

OHC_ENG = OHC_ENG.fit_transform(df[['Item_Type','Outlet_Type','Outlet_Location_Type']]) 

#Dropping
df.drop(['Item_Type','Outlet_Location_Type','Outlet_Type','Item_Fat_Content'],axis = 1 , inplace=True) 

# Concat all Variables
df = pd.concat([df,En,OHC_ENG],axis = 1 )
df.reset_index(drop=True)

df.head(5)

4.2. Comparing ML Algorithms Performance-using cross_val_score
* cross_var_score
* MinMax Scaler

In [ ]:
# Evaluating Best Model 

X = df.drop('Item_Outlet_Sales',axis= 1)
y = np.log1p(df["Item_Outlet_Sales"])

#Scaling 
MN = MinMaxScaler()
X = MN.fit_transform(X)

# Model initilization

Lnr = LinearRegression()
SVR_rbf = SVR(kernel = 'linear')
DT = DecisionTreeRegressor()
RDF = RandomForestRegressor()
XR = XGBRegressor()

# Model 

model_list = [Lnr,DT,RDF,XR,SVR_rbf]

def best_ml (model,X,y,cv=10): #Finding the Best model 
    cross = -cross_val_score(model,X,y,cv=cv,scoring='neg_mean_squared_error').mean()
    final = np.sqrt(cross)
    final_score.append(final)
    
final_score = []



for i in model_list:
    best_ml(i,X,y)

pd.DataFrame(data = final_score,
             columns = ['mean_Squared_error'],
             index = ['LinearRegression','SupVecMach','DecisionTree','RandomForest','XGBRegressor'])

4.3. Feature Importance

In [ ]:
from sklearn.inspection import permutation_importance
import shap

name = df.drop('Item_Outlet_Sales',axis = 1)
pd.DataFrame(data = X, columns =name.columns)
X = name
y = np.log1p(df["Item_Outlet_Sales"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
SVR_rbf = SVR(kernel = 'linear')

XGB = XGBRegressor()
XGB = XGB.fit(X_test,y_test)

perm_importance = permutation_importance(XGB, X_test, y_test)
explainer = shap.TreeExplainer(XGB)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values, X_test)

4.4. Tunnig XGBRegressor and SVR

In [ ]:
#splitting 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

SVR_rbf = SVR(kernel = 'linear')
XR = XGBRegressor()

#--------------------->   GridSearchCV   <-----------------------------------
param_grid = {'C' :[0.1,1,10,100,1000], 'gamma' : [1,0.1,0.01,0.001,0.0001]}
grid = GridSearchCV(SVR_rbf , param_grid , verbose = 3 , n_jobs = -1)
grid.fit(X_train,y_train)  # if you download the code please remove the # to find the best Parameter but it will take arround 20 min to finish the computation
SVR_y_pred = grid.predict(X_test)


#---------------------> Hyper Parameter Optimization <-----------------------
n_estimators = [100,500,900,1100,1500]
max_depth = [2,3,5,10,15]
booster = ['gbtree', 'gblinear']
learning_rate = [0.05,0.1,0.15,0.20]
min_child_weight = [1,2,3,4]
base_score = [0.25,0.5,0.75,1]

#-------------> Define the grid of Hyperparameters to search
hyperparameter_grid = { 'n_estimators': n_estimators,'max_depth': max_depth,'booster': booster,
                       'learning_rate': learning_rate,'min_child_weight': min_child_weight,
                       'base_score' : base_score}
random_cv = RandomizedSearchCV(estimator = XR,
                              param_distributions=hyperparameter_grid,
                              cv=5,n_iter = 50,
                              scoring = 'neg_mean_absolute_error',n_jobs = -1,
                              verbose = 5,
                              return_train_score=True,
                              random_state=42)

random_cv.fit(X_train,y_train)  # if you download the code please remove the # to find the best Parameter but it will take arround 20 min to finish the computation
XR_y_pred = random_cv.predict(X_test)

# -----------------> Best Parameters <---------------------
#grid..best_parameter_
#random_cv.best_parameter_


# -----------------> Final Results <------------------------

print("Mean Squared Error in SVM",np.sqrt(mean_squared_error(y_test,SVR_y_pred)))
print("Mean Squared Error in XGBoost",np.sqrt(mean_squared_error(y_test,XR_y_pred)))

Improvemen After Tunning only 0.3 reduction in Mean Square Error

In [ ]:
path="../input/loanimages/upvote.jpg"
display(Image.open(path))